本次作业以垃圾邮件分类任务为基础，要求提取文本特征并使用朴素贝叶斯算法进行垃圾邮件识别（调用已有工具包或自行实现）。

### 任务介绍
电子邮件是互联网的一项重要服务，在大家的学习、工作和生活中会广泛使用。但是大家的邮箱常常被各种各样的垃圾邮件填充了。有统计显示，每天互联网上产生的垃圾邮件有几百亿近千亿的量级。因此，对电子邮件服务提供商来说，垃圾邮件过滤是一项重要功能。而朴素贝叶斯算法在垃圾邮件识别任务上一直表现非常好，至今仍然有很多系统在使用朴素贝叶斯算法作为基本的垃圾邮件识别算法。

本次实验数据集来自[Trec06](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06)的中文垃圾邮件数据集，目录解压后包含三个文件夹，其中data目录下是所有的邮件（未分词），已分词好的邮件在data_cut目录下。邮件分为邮件头部分和正文部分，两部分之间一般有空行隔开。标签数据在label文件夹下，文件中每行是标签和对应的邮件路径。‘spam’表示垃圾邮件，‘ham’表示正常邮件。

本次实验

基本要求：
1. 提取正文部分的文本特征；
2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
4. 对比特征数目（词表大小）对模型效果的影响；
5. 提交代码和实验报告。

扩展要求：
1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
2. 尝试自行实现朴素贝叶斯算法细节；
3. 尝试对比不同的概率计算方法。

### 导入工具包

In [2]:
'''
提示：
若调用已有工具包，sklearn中提供了一些可能会用到的类。
'''
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV # 训练集测试集划分
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB,GaussianNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式

from sklearn.metrics import accuracy_score, precision_score, recall_score # 评估指标
RANDOM_SEED = 2024

In [3]:
#提取文件内容
import os
import pandas as pd
import time


def read_mail_files(data_folder_path):
    data_list = []
    file_count = 0
    max_files_to_read = 50000  # 设置要读取的最大文件数量
    start_time = time.time()  # 记录开始时间
    # 读取邮件内容
    for root, dirs, files in os.walk(data_folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    data_list.append({'mail_content': content, 'file_path': file_path})
            except UnicodeDecodeError:
                print(f"文件 {file_path} 编码格式有误，无法正确读取")
            file_count += 1
            if file_count % 1000 == 0:  # 每读取1000个文件进行一次时间打印
                elapsed_time = time.time() - start_time
                print(f"已读取 {file_count} 个文件，耗时 {elapsed_time:.2f} 秒")
            if file_count >= max_files_to_read:  # 判断是否达到要读取的最大文件数量，达到则停止读取
                break
        if file_count >= max_files_to_read:  # 如果当前文件夹遍历完已经达到数量，直接跳出外层循环
            break
    return pd.DataFrame(data_list)

# 假设data文件夹路径
data_folder_path = R'D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06c-utf8\data_cut'
# 假设label文件夹路径（标签文件所在文件夹）
label_folder_path = R'D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06c-utf8\label'

data_df = read_mail_files(data_folder_path)
data_df

已读取 1000 个文件，耗时 0.08 秒
已读取 2000 个文件，耗时 0.15 秒
已读取 3000 个文件，耗时 0.23 秒
已读取 4000 个文件，耗时 0.31 秒
已读取 5000 个文件，耗时 0.38 秒
已读取 6000 个文件，耗时 0.46 秒
已读取 7000 个文件，耗时 0.54 秒
已读取 8000 个文件，耗时 0.62 秒
已读取 9000 个文件，耗时 0.70 秒
已读取 10000 个文件，耗时 0.78 秒
已读取 11000 个文件，耗时 0.85 秒
已读取 12000 个文件，耗时 0.93 秒
已读取 13000 个文件，耗时 1.01 秒
已读取 14000 个文件，耗时 1.09 秒
已读取 15000 个文件，耗时 1.19 秒
已读取 16000 个文件，耗时 1.29 秒
已读取 17000 个文件，耗时 1.37 秒
已读取 18000 个文件，耗时 1.45 秒
已读取 19000 个文件，耗时 1.52 秒
已读取 20000 个文件，耗时 1.60 秒
已读取 21000 个文件，耗时 1.67 秒
已读取 22000 个文件，耗时 1.75 秒
已读取 23000 个文件，耗时 1.83 秒
已读取 24000 个文件，耗时 1.90 秒
已读取 25000 个文件，耗时 1.97 秒
已读取 26000 个文件，耗时 2.05 秒
已读取 27000 个文件，耗时 2.13 秒
已读取 28000 个文件，耗时 2.21 秒
已读取 29000 个文件，耗时 2.28 秒
已读取 30000 个文件，耗时 2.35 秒
已读取 31000 个文件，耗时 2.43 秒
已读取 32000 个文件，耗时 2.51 秒
已读取 33000 个文件，耗时 2.59 秒
已读取 34000 个文件，耗时 2.71 秒
已读取 35000 个文件，耗时 2.79 秒
已读取 36000 个文件，耗时 2.86 秒
已读取 37000 个文件，耗时 2.93 秒
已读取 38000 个文件，耗时 3.00 秒
已读取 39000 个文件，耗时 3.07 秒
已读取 40000 个文件，耗时 3.14 秒
已读取 41000 个文件，耗时 3.21 秒
已读取 42000 个文件，耗时 3.28 秒
已

,mail_content,file_path
0,Received: from hp-5e1fe6310264 ([218.79.188.13...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...
1,Received: from jdl.ac.cn ([159.226.42.8])\n\tb...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...
2,Received: from 163.con ([61.141.165.252])\n\tb...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...
3,Received: from 12.com ([222.50.6.150])\n\tby s...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...
4,Received: from dghhkjk.com ([59.36.183.208])\n...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...
...,...,...
49995,Received: from 12.com ([222.50.13.183])\n\tby ...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...
49996,Received: from mails.tsinghua.edu.cn (mails.ts...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...
49997,Received: from 163.com ([218.1.91.241])\n\tby ...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...
49998,Received: from mails.tsinghua.edu.cn (mails.ts...,D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06...


In [4]:
data_df

#删除data_df中file_path列
data_df = data_df.drop('file_path',axis=1)
#修改data_df，删除mail_content的前文，只保留正文内容，正文内容与前面用一个空行分隔

def extract_body_content(data_df):
    data_df['body_content'] = data_df['mail_content'].apply(lambda x: x.split('\n\n', 1)[-1] if '\n\n' in x else x)
    return data_df[['body_content']]

# 调用函数提取正文内容，得到只包含正文的新DataFrame
data_df = extract_body_content(data_df)
data_df

,body_content
0,[ 课 程 背 景 ]\n\n \n每 一位 管理 和 技术人员 都 清楚 地 ...
1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ， 跟 儿子 感情 不 和...
2,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！\n我 是 深圳 金海 实业 有...
3,贵 公司 负责人 ( 经理 / 财务 ） 您好 ：\n深圳市 华龙 公司 受 多家 公司 委...
4,这是 一封 HTML 格式 信件 ！\n\n- - - - - - - - - - - - ...
...,...
49995,尊敬 的 商家 朋友 您好 ：\n我 是 深圳市 昌隆 实业 有限公司 的 。 我司 实力雄...
49996,有些 感情 我们 很 容易 辨别 是 爱情\n有些 却 很 难\n爱情 没有 定义\n也 无...
49997,九 、 十月份 国 内 机 票 特 价 （ 不 含 建设费 ）\...
49998,片目 及 活动 时间 为 ：\n9 月 25 日 （ 周日 ） 19 ： 00 《 走进...


In [5]:
#读取标签
def read_first_10000_labels(label_file_path):
    label_list = []
    count = 0
    with open(label_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split(' ')  # 按空格分割每行获取标签和文件路径
            label = parts[0]
            label_list.append({'label': label})
            count += 1
            if count >= 50000:  # 达50000个标签后停止读取
                break
    return pd.DataFrame(label_list)

# 假设标签文件路径
label_file_path = R"D:\Code\yufengjihua\ML\实验\基于朴素贝叶斯垃圾邮件识别\trec06c-utf8\label\index" 
label_df = read_first_10000_labels(label_file_path)

#在label列中，将spam标签替换为1，ham标签替换为0

label_df['label'] = label_df.apply(lambda x: 1 if x['label'] == 'spam' else 0, axis=1)

label_df

,label
0,1
1,0
2,1
3,1
4,1
...,...
49995,1
49996,0
49997,1
49998,0


In [8]:
#特征提取
X = data_df['body_content']
y = label_df['label']

# 特征提取，使用词袋模型（CountVectorizer）将文本特征进行向量化
vectorizer = CountVectorizer()
# 对文本数据进行拟合和转换，得到数值特征矩阵
X_counter = vectorizer.fit_transform(X)

# 特征提取，使用TF-IDF模型（TfidfTransformer）将文本特征进行向量化
vectorizer = TfidfVectorizer()
X_Tfidf = vectorizer.fit_transform(X)

In [9]:
#划分训练集和测试集

x_train_counter, x_test_counter, y_train_counter, y_test_counter = train_test_split(X_counter, y, test_size=0.2,random_state=RANDOM_SEED)
x_train_Tfidf, x_test_Tfidf, y_train_Tfidf, y_test_Tfidf = train_test_split(X_Tfidf, y, test_size=0.2,random_state=RANDOM_SEED)

print(x_train_counter.shape)
print(x_test_counter.shape)
print(y_train_counter.shape)
print(y_test_counter.shape)

print(x_train_Tfidf.shape)
print(x_test_Tfidf.shape)
print(y_train_Tfidf.shape)
print(y_test_Tfidf.shape)

(40000, 173293)
(10000, 173293)
(40000,)
(10000,)
(40000, 173293)
(10000, 173293)
(40000,)
(10000,)


In [10]:
# 创建多项式朴素贝叶斯模型对象
model = MultinomialNB()

# 使用训练集数据进行模型训练
model.fit(x_train_Tfidf, y_train_Tfidf)

# 在测试集上进行预测
y_pred = model.predict(x_test_Tfidf)

# 计算准确率
accuracy = accuracy_score(y_test_Tfidf, y_pred)
# 计算精准率
precision = precision_score(y_test_Tfidf, y_pred)
# 计算召回率
recall = recall_score(y_test_Tfidf, y_pred)

print("准确率:", accuracy)
print("精准率:", precision)
print("召回率:", recall)

准确率: 0.9765
精准率: 0.9740659340659341
召回率: 0.991351028929317


In [11]:
# 创建伯努利朴素贝叶斯模型对象
model = BernoulliNB()

# 使用训练集数据进行模型训练
model.fit(x_train_counter, y_train_counter)

# 在测试集上进行预测
y_pred = model.predict(x_test_counter)

# 计算准确率
accuracy = accuracy_score(y_test_counter, y_pred)
# 计算精准率
precision = precision_score(y_test_counter, y_pred)
# 计算召回率
recall = recall_score(y_test_counter, y_pred)

print("准确率:", accuracy)
print("精准率:", precision)
print("召回率:", recall)

准确率: 0.941
精准率: 0.9823343848580441
召回率: 0.9287205487623024


In [12]:
# 创建多项式朴素贝叶斯模型对象
model = ComplementNB()

# 使用训练集数据进行模型训练
model.fit(x_train_Tfidf, y_train_Tfidf)

# 在测试集上进行预测
y_pred = model.predict(x_test_Tfidf)

# 计算准确率
accuracy = accuracy_score(y_test_Tfidf, y_pred)
# 计算精准率
precision = precision_score(y_test_Tfidf, y_pred)
# 计算召回率
recall = recall_score(y_test_Tfidf, y_pred)

print("准确率:", accuracy)
print("精准率:", precision)
print("召回率:", recall)

准确率: 0.9708
精准率: 0.9776586237712243
召回率: 0.9788249328959141


### 三种贝叶斯分类都在0.9以上